## Import

In [ ]:
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
from pathlib import Path

import beamtest_analysis_helper as helper
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import hist
import mplhep as hep
hep.style.use('CMS')

## Read track file

In [ ]:
df = pd.read_pickle('./desy_apr2024_track/track_681_R9C11_R8C11_R8C10_R8C11.pkl')
df

## Specify track information

In [ ]:
chip_labels = [0, 1, 2, 3]

## DESY TB Dec2023, Run39 configs
# chip_names = ["EPIR_Pair1", "BAR_4", "BAR_6", "BB_1-3"]
# high_voltages = [260, 260, 260, 200]
# offsets = [15, 10, 15, 10]
# chip_fignames = chip_names
# chip_figtitles = [
#     f"EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
#     f"Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
#     f"Barcelona 6 HV{high_voltages[2]}V OS:{offsets[2]}",
#     f"BB 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"
# ]

## DESY TB Feb2024, Run36 configs
# chip_names = ["EPIR_Pair1", "BAR_4", "BAR_5", "EPIR_Pair4"]
# high_voltages = [250, 260, 210, 200]
# offsets = [15, 20, 20, 15]
# chip_fignames = chip_names
# chip_figtitles = [
#     f"EPIR Pair1 HV{high_voltages[0]}V OS:{offsets[0]}",
#     f"Barcelona 4 HV{high_voltages[1]}V OS:{offsets[1]}",
#     f"Barcelona 5 HV{high_voltages[2]}V OS:{offsets[2]}",
#     f"EPIR Pair4 HV{high_voltages[3]}V OS:{offsets[3]}"
# ]

## DESY TB Apr2024, Run13 configs
chip_names = ["BAR_4", "BAR_12", "BAR_13", "BB_1-3"]
high_voltages = [260, 120, 120, 200]
offsets = [20, 20, 20, 20]
chip_fignames = chip_names
chip_figtitles = [
    f"Barcelona 4 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"Barcelona 12 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"Barcelona 13 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"BB 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"
]
fig_tags = [
    'R9C11',
    'R8C11',
    'R8C10',
    'R8C11',
]

## Set output directory depending on TB campaign

In [ ]:
current_dir = Path('./')
output_mother_dir = current_dir / 'etroc_TB_figs'
output_mother_dir.mkdir(exist_ok=True)

### Now you need change the directory name per campaign
### Naming rule is this:
### <TB location>_TB_MonthYear
### E.g. desy_TB_Apr2024, cern_TB_Sep2023, fnal_TB_Jul2024

output_campaign_dir = output_mother_dir / 'desy_TB_Apr2024'
output_campaign_dir.mkdir(exist_ok=True)

## Make 1D TDC hists

In [ ]:
h_track = helper.return_hist_pivot(input_df=df, chipNames=chip_names, board_id_to_analyze=chip_labels, hist_bins=[100, 128, 128])
for iboard in chip_labels:
    helper.plot_1d_TDC_histograms(h_track, chip_names[iboard], 'desy', chip_figtitles[iboard], slide_friendly=False, save_mother_dir=output_campaign_dir, tag=fig_tags[iboard])
del h_track

## Make TOA correlation plots

In [ ]:
helper.plot_TOA_correlation(df, board_id1=1, board_id2=2, boundary_cut=3, board_names=chip_names, draw_boundary=True, tb_loc='desy', save_mother_dir=output_campaign_dir)
helper.plot_TOA_correlation(df, board_id1=1, board_id2=3, boundary_cut=3, board_names=chip_names, draw_boundary=True, tb_loc='desy', save_mother_dir=output_campaign_dir)
helper.plot_TOA_correlation(df, board_id1=2, board_id2=3, boundary_cut=3, board_names=chip_names, draw_boundary=True, tb_loc='desy', save_mother_dir=output_campaign_dir)

## Apply TDC cuts

In [ ]:
tot_cuts = {}

for idx in [1, 2, 3]:
    lower_bound = df['tot'][idx].quantile(0.01)
    upper_bound = df['tot'][idx].quantile(0.96)
    tot_cuts[idx] = [round(lower_bound), round(upper_bound)]

tot_cuts

In [ ]:
## Selecting good hits with TDC cuts
tdc_cuts = {}
for idx in [1, 2, 3]:
    if idx == 1:
        tdc_cuts[idx] = [0, 1100, 250, 500, tot_cuts[idx][0], tot_cuts[idx][1]]
    else:
        tdc_cuts[idx] = [0, 1100, 0, 1100, tot_cuts[idx][0], tot_cuts[idx][1]]

track_tmp_df = helper.tdc_event_selection_pivot(df, tdc_cuts_dict=tdc_cuts)

In [ ]:
dut1_dut2_params, dut1_dut2_dist = helper.return_TOA_correlation_param(track_tmp_df, board_id1=1, board_id2=2)
dut1_ref_params, dut1_ref_dist = helper.return_TOA_correlation_param(track_tmp_df, board_id1=1, board_id2=3)
dut2_ref_params, dut2_ref_dist = helper.return_TOA_correlation_param(track_tmp_df, board_id1=2, board_id2=3)

condition1 = np.abs(dut1_dut2_dist) < 3*np.std(dut1_dut2_dist)
condition2 = np.abs(dut1_ref_dist) < 3*np.std(dut1_ref_dist)
condition3 = np.abs(dut2_ref_dist) < 3*np.std(dut2_ref_dist)

distance_condition = condition1 & condition2 & condition3

In [ ]:
analysis_df = track_tmp_df[distance_condition].reset_index(drop=True)

In [ ]:
h_track = helper.return_hist_pivot(input_df=analysis_df, chipNames=chip_names, board_id_to_analyze=chip_labels, hist_bins=[100, 128, 128])
for iboard in [1,2,3]:
    helper.plot_1d_TDC_histograms(h_track, chip_names[iboard], 'desy', chip_figtitles[iboard], slide_friendly=False, save_mother_dir=output_campaign_dir, tag=fig_tags[iboard]+'_afterTDCcuts')
del h_track

## Random sampling iteration

In [ ]:
analysis_df = analysis_df.reset_index().rename(columns={'index':'evt'})
n = int(0.75*analysis_df.shape[0])
indices = np.random.choice(analysis_df['evt'].unique(), n, replace=False)
tdc_filtered_df = analysis_df.loc[analysis_df['evt'].isin(indices)]

## Make dataframe in time

In [ ]:
board_to_analyze = [1,2,3]

d = {}

for idx in board_to_analyze:
    bins = 3.125/tdc_filtered_df['cal'][idx].mean()
    d[f'toa_b{str(idx)}'] = (12.5 - tdc_filtered_df['toa'][idx] * bins)*1e3
    d[f'tot_b{str(idx)}'] = ((2*tdc_filtered_df['tot'][idx] - np.floor(tdc_filtered_df['tot'][idx]/32)) * bins)*1e3

df_in_time = pd.DataFrame(data=d)
del d, tdc_filtered_df

In [ ]:
df_in_time

## TOA hists before TWC

In [ ]:
h_before_twc_toa1 = hist.Hist(hist.axis.Regular(50, 0, 14000, name='toa_b0', label='toa_b0'))
h_before_twc_toa2 = hist.Hist(hist.axis.Regular(50, 0, 14000, name='toa_b1', label='toa_b1'))
h_before_twc_toa3 = hist.Hist(hist.axis.Regular(50, 0, 14000, name='toa_b2', label='toa_b2'))

h_before_twc_toa1.fill(df_in_time['toa_b1'])
h_before_twc_toa2.fill(df_in_time['toa_b2'])
h_before_twc_toa3.fill(df_in_time['toa_b3'])

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(25, 10))
yerr_bool = np.array(h_before_twc_toa1.values(), dtype=bool)
hep.histplot(h_before_twc_toa1, ax=axes[0], yerr=yerr_bool)
axes[0].set_xlabel('TOA1')

yerr_bool = np.array(h_before_twc_toa2.values(), dtype=bool)
hep.histplot(h_before_twc_toa2, ax=axes[1], yerr=yerr_bool)
axes[1].set_xlabel('TOA2')

yerr_bool = np.array(h_before_twc_toa3.values(), dtype=bool)
hep.histplot(h_before_twc_toa3, ax=axes[2], yerr=yerr_bool)
axes[2].set_xlabel('TOA3')

plt.tight_layout()

### Delta TOA hists before TWC

In [ ]:
board_list = [1, 2, 3]

del_toa_b0 = (0.5*(df_in_time[f'toa_b{board_list[1]}'] + df_in_time[f'toa_b{board_list[2]}']) - df_in_time[f'toa_b{board_list[0]}'])
del_toa_b1 = (0.5*(df_in_time[f'toa_b{board_list[0]}'] + df_in_time[f'toa_b{board_list[2]}']) - df_in_time[f'toa_b{board_list[1]}'])
del_toa_b2 = (0.5*(df_in_time[f'toa_b{board_list[0]}'] + df_in_time[f'toa_b{board_list[1]}']) - df_in_time[f'toa_b{board_list[2]}'])

In [ ]:
h_before_twc_delta_toa1 = hist.Hist(hist.axis.Regular(50, -2500, 2500, name='toa_b1', label='toa_b1'))
h_before_twc_delta_toa2 = hist.Hist(hist.axis.Regular(50, -2500, 2500, name='toa_b2', label='toa_b2'))
h_before_twc_delta_toa3 = hist.Hist(hist.axis.Regular(50, -2500, 2500, name='toa_b3', label='toa_b3'))

h_before_twc_delta_toa1.fill(del_toa_b0)
h_before_twc_delta_toa2.fill(del_toa_b1)
h_before_twc_delta_toa3.fill(del_toa_b2)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(25, 10))
yerr_bool = np.array(h_before_twc_delta_toa1.values(), dtype=bool)
hep.histplot(h_before_twc_delta_toa1, ax=axes[0], yerr=yerr_bool)
axes[0].set_xlabel('0.5*(TOA2+TOA3)-TOA1', fontsize=15)


yerr_bool = np.array(h_before_twc_delta_toa2.values(), dtype=bool)
hep.histplot(h_before_twc_delta_toa2, ax=axes[1], yerr=yerr_bool)
axes[1].set_xlabel('0.5*(TOA1+TOA3)-TOA2', fontsize=15)


yerr_bool = np.array(h_before_twc_delta_toa3.values(), dtype=bool)
hep.histplot(h_before_twc_delta_toa3, ax=axes[2], yerr=yerr_bool)
axes[2].set_xlabel('0.5*(TOA2+TOA3)-TOA1', fontsize=15)

plt.tight_layout()

### Pairwise delta TOA before TWC

In [ ]:
pre_diffs = {}
for board_a in board_to_analyze:
    for board_b in board_to_analyze:
        if board_b <= board_a:
            continue
        name = f"{board_a}{board_b}"
        pre_diffs[name] = np.asarray(df_in_time[f'toa_b{board_a}'] - df_in_time[f'toa_b{board_b}'])

fit_params = {}
scores = []

for ikey in pre_diffs.keys():
    params, eval_scores = helper.fwhm_based_on_gaussian_mixture_model(pre_diffs[ikey], 'desy', n_components=3, show_sub_gaussian=False, show_fwhm_guideline=False, show_number=True, tag=ikey, save_mother_dir=output_campaign_dir, fname_tag='before_TWC')
    fit_params[ikey] = float(params[0]/2.355)
    scores.append(eval_scores)

print(scores)

In [ ]:
keys = list(pre_diffs.keys())
resolutions = helper.return_resolution_three_board_fromFWHM(fit_params, var=keys, board_list=board_to_analyze)
resolutions

### Investigate TWC

In [ ]:
helper.plot_TWC(input_df=df_in_time, board_list=board_to_analyze, tot_range=[1500, 6500], tb_loc='desy', poly_order=2, save_mother_dir=output_campaign_dir, print_func=False)

### Iterative TWC step by step

In [ ]:
corr_toas = helper.three_board_iterative_timewalk_correction(df_in_time, iterative_cnt=2, poly_order=2, board_list=board_to_analyze)

In [ ]:
helper.plot_TWC(input_df=df_in_time, board_list=board_to_analyze, tot_range=[2000, 6500], poly_order=2, corr_toas=corr_toas)

In [ ]:
pre_diffs = {}
for board_a in board_to_analyze:
    for board_b in board_to_analyze:
        if board_b <= board_a:
            continue
        name = f"{board_a}{board_b}"
        pre_diffs[name] = np.asarray(corr_toas[f'toa_b{board_a}'] - corr_toas[f'toa_b{board_b}'])

fit_params = {}
scores = []

for ikey in pre_diffs.keys():
    params, eval_scores = helper.fwhm_based_on_gaussian_mixture_model(pre_diffs[ikey], n_components=3, show_sub_gaussian=False, show_fwhm_guideline=False, show_number=True, title=ikey, save_mother_dir=output_campaign_dir, fname_tag='after_TWC')
    fit_params[ikey] = float(params[0]/2.355)
    scores.append(eval_scores)

print(scores)

In [ ]:
keys = list(pre_diffs.keys())
resolutions = helper.return_resolution_three_board_fromFWHM(fit_params, var=keys, board_list=board_to_analyze)
resolutions